In [ ]:
import opensimplex
import numpy as np
import holoviews as hv

hv.extension("bokeh","matplotlib",logo=False)

o={"cmap":"gist_gray"}




x1=np.linspace(0,2.,250)
x2=np.linspace(0.,2.,250)
val1=opensimplex.noise2array(x1,x2)
val2=opensimplex.noise2array(x1*2,x2*2)
val3=opensimplex.noise2array(x1*4,x2*4)
   
o={"cmap":"gist_gray"}


oneField=(hv.Image(val1,bounds=(0,0,1,1)).opts(**o)+\
          hv.Image(val2,bounds=(0,0,1,1)).opts(**o)+\
          hv.Image(val3,bounds=(0,0,1,1)).opts(**o)).opts(toolbar=None)
          
hv.renderer("bokeh").save(oneField,"oneField",fmt="png")  
oneField

In [ ]:

totField=hv.Image(val1+val2*.5+val3*.25,bounds=(0,0,1,1)).opts(**o).opts(toolbar=None)
hv.renderer("bokeh").save(totField,"totField",fmt="png")  
totField

In [ ]:
import syntheticModel
hv.extension("bokeh","matplotlib",logo=False)


In [ ]:
p1 = hv.Image(val1,bounds=(0,0,1,1)).opts(**o,toolbar=None)
p2 = hv.Image(val1,bounds=(0,0,1,1)).opts(**o,toolbar=None)
p3 = hv.Image(val1,bounds=(0,0,1,1)).opts(**o,toolbar=None)
m=(p2+p3).opts(toolbar=None)
n=hv.Layout(m+p1).cols(2)


def createPlot(data,p1,p2,p3):
    if isinstance(data,np.ndarray):
        fld=data
        hyper=syntheticModel.hypercube(ns=[data.shape[2],data.shape[1],data.shape[0]])
    else:
        fld=data.getNdArray()
        hyper=data.getHyper()
    ax1=hyper.axes[0]
    ax2=hyper.axes[1]
    ax3=hyper.axes[2]
    mn=fld.min()
    mx=fld.max()
    front=fld[p3,:,::-1].T
    top=fld[:,:,p1]
    side=fld[:,p2,::-1].T
    b=(ax1.o,ax1.o+ax1.d*(ax1.n-1),ax2.o,ax2.o+ax2.d*(ax2.n-1))
    print(b,mn,mx)
    frontF=hv.Image(front,kdims=["X","Z"],bounds=(ax2.o,ax1.o,ax2.o+ax2.d*(ax2.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_gray",clim=(mn,mx),toolbar=None,invert_yaxis=True,aspect=1,axiswise=True)
    sideF=hv.Image(side,kdims=["Y","Z"],bounds=(ax3.o,ax1.o,ax3.o+ax3.d*(ax3.n-1),ax1.o+ax1.d*(ax1.n-1))).opts(\
             cmap="gist_gray",clim=(mn,mx),toolbar=None,yaxis=None,invert_yaxis=True,aspect=1.,axiswise=True)
    topF=hv.Image(top,kdims=["X","Y"],bounds=(ax1.o,ax2.o,ax1.o+ax1.d*(ax1.n-1),ax2.o+ax2.d*(ax2.n-1))).opts(\
             cmap="gist_gray",clim=(mn,mx),toolbar=None,xaxis=None,aspect=1,axiswise=True)
    botF=hv.Layout(frontF+sideF).opts(toolbar=None)
    tot=hv.Layout(topF+hv.Empty()+frontF+sideF).cols(2).opts(toolbar=None)
    return tot
    
    
        

In [ ]:
import syntheticModel
hv.extension("bokeh","matplotlib",logo=False)

hyper=syntheticModel.hypercube(ns=[20,400,200],os=[0,0,0],ds=[4,4,4])
model=syntheticModel.geoModel(hyper,{"vel":5000.},"vel")
depositC=syntheticModel.deposit()
depositI=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.)
depositN=syntheticModel.deposit(noiseDev=300.)
depositIC=syntheticModel.deposit(interbedThick=5.,interbedThickDev=3.,interbedPropDev=200.,interbedDev=8.)
mt=depositI.apply(model,thick=40,prop=4700)
mt2=depositI.apply(mt,thick=70,prop=4100)
mt3=depositI.apply(mt2,thick=60,prop=3700)
mt4=depositI.apply(mt3,thick=60,prop=3100,expand=False)
mt5=syntheticModel.squish().apply(mt4,begFreq=13,maxShift=150)
mt6=depositI.apply(mt5,thick=30)

In [ ]:

squishP=createPlot(mt6.get(),200,200,100)
squishP

In [ ]:

mt6=depositI.apply(mt5,thick=30)
xs=np.random.uniform(low=.2,high=.8,size=(10,))
ys=np.random.uniform(low=.2,high=.8,size=(10,))
zs=np.random.uniform(low=.3,high=.8,size=(10,))
angles=np.random.uniform(low=45,high=46,size=(10,))
rads=np.random.uniform(low=1850,high=2150,size=(10,))
shifts=np.random.uniform(low=100,high=140,size=(10,))
lengths=np.random.uniform(low=170,high=240,size=(10,))
#azs=np.random.uniform(low=-1,high=2,size=(10,))
xs[0]=.5
ys[0]=.5
zs[0]=.5
shifts[0]=30
for i in range(1):
    mt6=syntheticModel.fault().apply(mt4,angle=angles[i],radius=rads[i],begx=xs[i],begy=ys[i],\
                            azimuth=45,\
        begz=zs[i],indicateI=True,shift=shifts[i],ruptureLength=lengths[i])


In [ ]:
faultsP=createPlot(mt6.get(),100,200,100)
#hv.renderer("bokeh").save(faultsP,"faults",fmt="png")
faultsP

In [ ]:
indicatorF=createPlot(mt6.getIntField("indicator"),200,200,100)
hv.renderer("bokeh").save(indicatorF,"indicatorF",fmt="png")
indicatorF
x=mt6.getIntField("indicator").getNdArray()


In [ ]:

mt6=depositI.apply(mt5,thick=30)
mt7=syntheticModel.salt().apply(mt6,seed=23,beg_freq=3.5,\
            salt_value={"vel":4500})

In [ ]:
saltP=createPlot(mt7.get(),200,200,100)
hv.renderer("bokeh").save(saltP,"saltP",fmt="png")
saltP

In [ ]:
mt6=depositI.apply(mt5,thick=30)
mt7=syntheticModel.chimneyBig().apply(mt6,width=.07)
mt8=depositC.apply(mt7,prop=1500,thicl=20)

In [ ]:
chimneyP=createPlot(mt8.get(),32,200,100)
hv.renderer("bokeh").save(chimneyP,"chimneyP",fmt="png")
chimneyP

In [ ]:
mt9=syntheticModel.salt().apply(mt8,beg_freq=3.5,\
            salt_value={"vel":4500},applyHills=True,maxShift=40)
saltP=createPlot(mt9.get(),31,200,100)
#hv.renderer("bokeh").save(riverP,"riverP",fmt="png")
saltP

In [ ]:
riverP=createPlot(mt9.get(),31,200,100)
hv.renderer("bokeh").save(riverP,"riverP",fmt="png")
riverP